<a href="https://colab.research.google.com/github/KosukhaOlexandr/reactions_prediction/blob/main/reactions_prediction_model_without_stopwords.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import pandas as pd

with open('/content/drive/MyDrive/reaction_prediction/concatenated_pos_neg.csv') as f:
  data = pd.read_csv(f, index_col = 0)

In [ ]:
data.head()

,msg_text,reaction_type
0,чий борщ у п’ятницю липня юнеско вирішить долю...,1
1,зруйнований міст у демидові на київщині планую...,1
2,безперечно головне слово сьогодні зміїний мабу...,1
3,прокремлівське видання общественная служба нов...,1
4,продовжується відвантаження боєприпасів продов...,1




---



In [ ]:
with open('/content/drive/MyDrive/reaction_prediction/stopwords.txt') as f:
  stopwords = pd.read_table(f, header = None)

In [ ]:
stopwords = stopwords.squeeze()

# Models using the stopwords extractioned version of the data

## Naive Bayes

### Count Vectorizer

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
count_vec = CountVectorizer()
bow = count_vec.fit_transform(data.msg_text)
#bow = np.array(bow.todense())
bow

<63494x119493 sparse matrix of type '<class 'numpy.int64'>'
	with 2479084 stored elements in Compressed Sparse Row format>

In [ ]:
X = bow
Y = data.reaction_type

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.20, stratify = Y, random_state = 42)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB(alpha = 0.01).fit(X_train, Y_train)

In [ ]:
Y_pred_test = model.predict(X_test)
Y_pred_train = model.predict(X_train)

In [ ]:
from sklearn.metrics import accuracy_score

print('Test set accuracy:', accuracy_score(Y_test, model.predict(X_test)))
print('Train set accuracy:', accuracy_score(Y_train, model.predict(X_train)))

Test set accuracy: 0.8180171667060399
Train set accuracy: 0.9152278767595236


In [ ]:
from sklearn.metrics import f1_score

print('Test set F1 score:', f1_score(Y_test, Y_pred_test))
print('Train set F1 score:', f1_score(Y_train, Y_pred_train))

Test set F1 score: 0.8431838230304675
Train set F1 score: 0.9256458074319658


### TF-IDF

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

tf_idf_vec = TfidfVectorizer()
bow_ti = tf_idf_vec.fit_transform(data.msg_text)
#bow_ti = np.array(bow_ti.todense())
bow_ti

<63494x119493 sparse matrix of type '<class 'numpy.float64'>'
	with 2479084 stored elements in Compressed Sparse Row format>

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

X = bow_ti
Y = data.reaction_type

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.30, stratify = Y, random_state = 42)

model = MultinomialNB(alpha = 0.01).fit(X_train, Y_train)

Y_pred = model.predict(X_test)
Y_pred_train = model.predict(X_train)

In [ ]:
from sklearn.metrics import accuracy_score

print('Test set accuracy:', accuracy_score(Y_test, Y_pred))
print('Train set accuracy:', accuracy_score(Y_train, Y_pred_train))

Test set accuracy: 0.8215129403118274
Train set accuracy: 0.9304758690516368


In [ ]:
from sklearn.metrics import f1_score

print('Test set F1 score:', f1_score(Y_test, Y_pred))
print('Train set F1 score:', f1_score(Y_train, Y_pred_train))

Test set F1 score: 0.853042876901798
Train set F1 score: 0.9408227363260303


##SVC

### TF-IDF

In [ ]:
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

X = bow_ti
Y = data.reaction_type

X_train_ti, X_test_ti, Y_train_ti, Y_test_ti = train_test_split(X, Y, test_size = 0.30, stratify = Y, random_state = 42)

svc_tf_idf = make_pipeline(StandardScaler(with_mean=False), SVC(C = 1, gamma='auto'))
svc_tf_idf.fit(X_train_ti, Y_train_ti)

Pipeline(steps=[('standardscaler', StandardScaler(with_mean=False)),
                ('svc', SVC(C=1, gamma='auto'))])

In [ ]:
Y_pred_test = svc_tf_idf.predict(X_test_ti)
Y_pred_train = svc_tf_idf.predict(X_train_ti)

In [ ]:
from sklearn.metrics import accuracy_score

print('Test set accuracy:', accuracy_score(Y_test_ti, Y_pred_test))
print('Train set accuracy:', accuracy_score(Y_train_ti, Y_pred_train))

Test set accuracy: 0.8065515250144365
Train set accuracy: 0.9779502756215547


In [ ]:
from sklearn.metrics import f1_score

print('Test set F1 score:', f1_score(Y_test_ti, Y_pred_test))
print('Train set F1 score:', f1_score(Y_train_ti, Y_pred_train))

Test set F1 score: 0.850306698622903
Train set F1 score: 0.9809952294147306


### Count Vectorizer

In [ ]:
X = bow
Y = data.reaction_type

In [ ]:
X_train_cv, X_test_cv, Y_train_cv, Y_test_cv = train_test_split(X, Y, test_size = 0.30, stratify = Y, random_state=42)

svc_cv = make_pipeline(StandardScaler(with_mean=False), SVC(C = 1, gamma='auto'))
svc_cv.fit(X_train_cv, Y_train_cv)

Pipeline(steps=[('standardscaler', StandardScaler(with_mean=False)),
                ('svc', SVC(C=1, gamma='auto'))])

In [ ]:
Y_pred_test = svc_cv.predict(X_test_cv)
Y_pred_train = svc_cv.predict(X_train_cv)

In [ ]:
print('Test set accuracy:', accuracy_score(Y_test_cv, Y_pred_test))
print('Train set accuracy:', accuracy_score(Y_train_cv, Y_pred_train))

Test set accuracy: 0.8154233818048191
Train set accuracy: 0.9680953988075149


In [ ]:
print('Test set F1 score:', f1_score(Y_test_cv, Y_pred_test))
print('Train set F1 score:', f1_score(Y_train_cv, Y_pred_train))

Test set F1 score: 0.8536707174962543
Train set F1 score: 0.9724702958763688
